In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Set MLflow tracking URI
mlflow.set_tracking_uri("http://localhost:5000")

In [ ]:
df = pd.read_csv("../data/data.csv")

X = df[["Risk_Score", "Transaction_Amount"]]
y = df["Fraud_Label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

In [ ]:
experiment_name = "my_experiment"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="rf-run"):
    # Hyperparameters
    n_estimators = 100
    max_depth = 5

    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, n_jobs = 1, random_state=42)
    rf.fit(X_train, y_train)

    # Evaluate
    accuracy = rf.score(X_test, y_test)

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_metric("accuracy", accuracy)

    mlflow.sklearn.log_model(rf, artifact_path="model", registered_model_name="my_sklearn_model")

    # from mlflow.tracking import MlflowClient
    # client = MlflowClient()
    # client.transition_model_version_stage(
    #     name="my_sklearn_model",
    #     version=1,
    #     stage="Production"
    # )

    print(f"Model logged with accuracy: {accuracy}")